In [ ]:
from pathlib import Path
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import numpy as np
import tensorflow as tf

from utils import structures_to_df, read_json_structures


In [7]:
df_public, df_private = structures_to_df()
df_public = df_public.merge(read_json_structures(Path('../data/dichalcogenides_public/defects')).rename({'structure': 'diff'}, axis=1), on=['_id'])
df_private = df_private.merge(read_json_structures(Path('../data/dichalcogenides_private/defects')).rename({'structure': 'diff'}, axis=0), on=['_id'])

2967it [00:18, 157.65it/s]
2966it [00:19, 151.92it/s]
2966it [00:01, 1723.35it/s]
2967it [00:01, 1824.46it/s]


In [115]:
def energy_within_threshold(prediction, target):
    e_thresh = 0.02
    error_energy = tf.math.abs(target - prediction)

    success = tf.math.count_nonzero(error_energy < e_thresh)
    total = tf.size(target)
    return success / tf.cast(total, tf.int64)


In [119]:
nfeat_bond = 100
r_cutoff = 20  # ключевое
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)

model = MEGNetModel(graph_converter=graph_converter,
                    centers=gaussian_centers,
                    width=gaussian_width,
                    metrics=energy_within_threshold)

model.train(df_public['diff'], df_public['band_gap'], epochs=100)

/Users/tomatoparetogmail.com/Desktop/Github/idao_22/env/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
24/24 [==============================] - 16s 65ms/step - loss: 0.4263 - energy_within_threshold: 0.0234
Epoch 2/100
24/24 [==============================] - 2s 61ms/step - loss: 0.2277 - energy_within_threshold: 0.0026
Epoch 3/100
24/24 [==============================] - 2s 65ms/step - loss: 0.0610 - energy_within_threshold: 0.0599
Epoch 4/100
24/24 [==============================] - 2s 72ms/step - loss: 0.0294 - energy_within_threshold: 0.0596
Epoch 5/100
24/24 [==============================] - 2s 74ms/step - loss: 0.0195 - energy_within_threshold: 0.0906
Epoch 6/100
24/24 [==============================] - 2s 72ms/step - loss: 0.0115 - energy_within_threshold: 0.1433
Epoch 7/100
24/24 [==============================] - 2s 68ms/step - loss: 0.0067 - energy_within_threshold: 0.2351
Epoch 8/100
24/24 [==============================] - 2s 70ms/step - loss: 0.0055 - energy_within_threshold: 0.3381
Epoch 9/100
24/24 [==============================] - 2s 68ms/step - loss: 0.005

In [139]:
model.train(df_public['diff'], df_public['band_gap'], epochs=20)

Epoch 1/20
24/24 [==============================] - 2s 63ms/step - loss: 4.0469e-04 - energy_within_threshold: 0.9424
Epoch 2/20
24/24 [==============================] - 2s 64ms/step - loss: 3.7199e-04 - energy_within_threshold: 0.9561
Epoch 3/20
24/24 [==============================] - 1s 60ms/step - loss: 3.7373e-04 - energy_within_threshold: 0.9564
Epoch 4/20
24/24 [==============================] - 2s 62ms/step - loss: 3.8546e-04 - energy_within_threshold: 0.9538
Epoch 5/20
24/24 [==============================] - 2s 77ms/step - loss: 4.0289e-04 - energy_within_threshold: 0.9397
Epoch 6/20
24/24 [==============================] - 2s 73ms/step - loss: 4.0056e-04 - energy_within_threshold: 0.9444
Epoch 7/20
24/24 [==============================] - 2s 71ms/step - loss: 3.8655e-04 - energy_within_threshold: 0.9490
Epoch 8/20
24/24 [==============================] - 2s 71ms/step - loss: 4.4836e-04 - energy_within_threshold: 0.9097
Epoch 9/20
24/24 [==============================] - 2s 7

In [141]:
model.save_model('09598tr_089lb.hdf5')
model.load_weights('09598tr_089lb.hdf5')

In [146]:
model.predict_structures(df_private['structure_y'])

array([[0.36183706],
       [1.1448069 ],
       [1.7996942 ],
       ...,
       [1.1438843 ],
       [0.387114  ],
       [0.3886133 ]], dtype=float32)

In [145]:
df_private['predictions']

0       0.361837
1       1.144807
2       1.799694
3       1.136510
4       0.405438
          ...   
2962    1.143082
2963    0.353465
2964    1.143884
2965    0.387114
2966    0.388613
Name: predictions, Length: 2967, dtype: float32

In [140]:
df_private['predictions'] = model.predict_structures(df_private['structure_y'])
df_private[['_id', 'predictions']].rename({'_id': 'id'}, axis=1).set_index('id').to_csv('megnet.csv')
# df_private